<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:
* An input layer is the information that you feed in to a system.  Whatever the data is.  This is what the neural network will use to determine outputs.
### Hidden Layer:
* A hidden layer in a neural network is a layer that is doing some work on the inputs, but cannot be seen or desribed by looking at it.  They are the steps between the input and output of a network.  It uses weights and biases to perform work on the data. The outputs are also determined by a given activation function of the network.
### Output Layer:
* The output layer are the predictions or classification of the data after it has gone through the network. As stated previously, it is helped to be determined by the activation function used in the neural network.
### Neuron:
* A Neuron (or node) in a neural network is the what the basic make-up of the network is. Each input, bias, hidden node, and output is a Neuron in the network.  Things get passed on, again, determined by the activation function through each layer, to determine the final Neurons (outputs).
### Weight:
* A weight is something that is attached to an input of the network.  They are set at the beginnning and adjusted throughout the training period of the network.  Weights essentially determine how influential an input is in determining the ouput.
### Activation Function:
* Have been talknig a lot about activation functions.  Their main function .. tee hee .. is to deal with non-linear data.  It can shrink information down to lower dimensions to then get a number that the network can use to determine the output.  In a classification case, the network uses the number produced by the activation function to determine what class the input is.
### Node Map:
![Neural Network Zoo](http://www.asimovinstitute.org/wp-content/uploads/2016/09/neuralnetworks.png)
### Perceptron:
* A Perceptron is one of the more basic neural networks.  It is typically used to classify binary outputs. It consists of 1 working neuron, that takes information from data (inputs) and tosses it out so that the activation function can determine it's value.

## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### A neural network begins with the inputs. Each input has a corresponding weight that is given to it initially.  The weights can be arbitrary at first.  The inputs are then fed into the neurons that perform work on the data.  The goal for the layers is to adjust the weights in order to minimize the error of predictions from real data. The next step is for the processed data to go through the activation fucntion. Which is a function that is able to deal with multi-dimensional data and allow the network to determine the class it will be in the output.  |

## Write your own perceptron code that can correctly classify a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [1]:
import pandas as pd
import numpy as np

inputs = np.array([
    [0, 0, 1],
    [1, 0, 1],
    [0, 1, 1],
    [1, 1, 1]
])

ground_truth = [[1], [1], [1], [0]]

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1-sx)

weights = 2 * np.random.random((3,1)) - 1

In [2]:
weights

array([[-0.39708544],
       [-0.98368861],
       [ 0.40078636]])

In [3]:
for iteration in range(10000):
    weighted_sum = np.dot(inputs, weights)
    
    activated_outputs = sigmoid(weighted_sum)
    
    error = ground_truth - activated_outputs
    
    adjustments = error * sigmoid_derivative(activated_outputs)
    
    weights += np.dot(inputs.T, adjustments)
    
print("Weights after training: ", weights)
print("Output after training: ", activated_outputs)

Weights after training:  [[-11.84095422]
 [-11.84095422]
 [ 17.81029548]]
Output after training:  [[0.99999998]
 [0.99745034]
 [0.99745034]
 [0.00281068]]


## Implement your own Perceptron Class and use it to classify a binary dataset like: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 
- [Titanic](https://raw.githubusercontent.com/ryanleeallred/datasets/master/titanic.csv)
- [A two-class version of the Iris dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/Iris.csv)

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [257]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from random import randrange

diabetes_df = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv")

In [258]:
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [259]:
X = diabetes_df.drop(columns=["Outcome"]).values
y = diabetes_df["Outcome"].values

X_train, X_test, y_train, t_test = train_test_split(X, y, test_size=.25)

In [260]:
# Make a prediction with weights

def predict(row, weights, bias):
    activation = bias
    for i in range(len(row)):
        activation += weights[i] * row[i]
    return 1.0 if activation >= 0.0 else 0.0

In [261]:
# Estimate Perceptron weights 
def train_weights(X_train, y_train, l_rate, n_epoch):
    
    #initialize weights and bias w/ value = 0
    weights = [0.0 for i in range(len(X_train[0]))]
    bias = 0.0
    
    #initialize min_error as inifinite
    min_error = float("inf")
    
    #iterate n_epoch times:
    for epoch in range(n_epoch):
        
        #initialize sum squared error for this iteration as 0
        sum_error = 0.0
        
        #iterate thru each row in X_train
        for i in range(len(X_train)):
            row = X_train[i]
            
            #call the predict function to get the predicted outcome for each row
            prediction = predict(row, weights, bias)
            
            #calc the error
            error = y_train[i] - prediction
            
            #update the sum squared error for this iteration w/ current weights and bias
            sum_error += error**2
            
            #update the bias
            bias += l_rate * error
            
            #iterate thru each feature and update the corresponding weight:
            for j in range(len(row)):
                weights[j] += l_rate * error * row[j]
                
        #if the sum squared error is less than prev minimum, store the weights, bias, and min_error in        
        if sum_error < min_error:
            min_error = sum_error
            best = [weights, bias]  #, min_error]
    return best                

In [262]:
# evaluate algorithm
best = train_weights(X_train, y_train, .01, 5000)

In [263]:
best

[[10.130000000039551,
  4.380000000002776,
  -1.3899999999927362,
  0.18000000000005395,
  0.8199999999987206,
  3.0539999999976604,
  159.45577000004067,
  -0.3399999999997484],
 -493.7899999996959]

In [264]:
#grab the weights and bias from the best variable
weights = best[0]
bias = best[1]

#generate predictions on the withheld test data using those weights and bias, row by row:
y_pred = []
for i in range(len(X_test)):
    row = X_test[i]
    y_pred.append(predict(row, weights, bias))

In [265]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))

Accuracy: 0.40625
Precision: 0.31901840490797545
Recall: 0.9454545454545454


## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?